In [1]:
import numpy as np
import scipy as sp

import plotly.plotly as ply
import plotly.graph_objs as go

import siggen
import onestage
from likelihood import *

In [2]:
#input parameters
a = siggen.Antenna()
a.load("/home/dima/.landing/AFS32.ini")
f0 = 120e6
N = 2 ** 16
fs = f0 * N / (N/2 -2);
f_res = fs / N
df = f_res
target_pel = np.array([0,0])

In [3]:
#calculating antenna parameters
beam_width = onestage.beam_width_est(antenna=a, f0=f0, df=df, fs=fs, f_res=f_res)
print(np.rad2deg(beam_width))

[ 67.8  31.5]


In [4]:
#Грубая оценка местоположения
size = np.array([np.ceil(np.pi / beam_width[1]) * 3, np.ceil(2 * np.pi / beam_width[0]) * 3], np.int)
R = 1000
target = np.array([[R*np.cos(target_pel[1])*np.cos(target_pel[0])],
                   [R*np.cos(target_pel[1])*np.sin(target_pel[0])],
                   [R*np.sin(target_pel[1])]])
lh = onestage.arp(a, f0, df, fs, f_res, size)
N = int(fs/f_res)
n_stop = int(np.ceil((f0 + df/2) / f_res))
n_start = int(np.ceil((f0 - df/2) / f_res))
s = siggen.wgn_baseband(a, target, N, fs)
alpha_line = np.array([np.linspace(0, 2 * np.pi, size[1], endpoint=False)])
alpha_mat = np.matmul(np.ones((1, size[0], 1)), alpha_line)

betta_line = np.transpose(np.array([np.linspace(-np.pi / 2, np.pi / 2, size[0])]))
betta_mat = np.matmul(betta_line, np.ones((1, 1, size[1])))

peleng = np.concatenate((alpha_mat, betta_mat), axis=0)
likelihood = np.zeros(size)

lh = Pel_Lh(s[:, n_start:n_stop], a, f0)
for i in range(size[0]):
    for j in range(size[1]):
        likelihood[i, j] = -lh.calc(peleng[:, i, j])

In [5]:
alpha = np.linspace(0, 2 * np.pi, size[1])
betta = np.linspace(-np.pi / 2 , np.pi / 2, size[0])
alpha_grid, betta_grid = np.meshgrid(alpha, betta)

trace = go.Surface(    x=np.rad2deg(alpha_grid),
                       y=np.rad2deg(betta_grid),
                       z=likelihood)
data = [trace]
ply.iplot(data, filename='Antenna radiaton pattern Cartesian')

In [13]:
max_ind = np.argmax(likelihood)
betta_max = max_ind % likelihood.shape[0]
alpha_max = max_ind % likelihood.shape[1]
pel_max = np.array([alpha_max, betta_max])

pel_est = sp.optimize.minimize(lh.calc, pel_max, method = 'Nelder-Mead')

In [15]:
pel_est.x

array([ 0.00098379, -0.00737459])